<a href="https://colab.research.google.com/github/FedorTaggenbrock/data_intensive_systems/blob/main/notebooks/main_tests_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Handle appending the right stuff to the path
import sys
sys.path.append('..')  # add parent directory to system path

# Import all modules (should be only our self-written ones)
# from clustering import jaccard_distance
# etc for all modules



ModuleNotFoundError: ignored

In [ ]:
!python ../main_tests.py

test


**Pip install stuff that needs to be installed**  
(for colab only; cannot be done inside the modules, would be stupid)

In [6]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=2952707ba7b5823e1fa8780ec50b9994837a1c5d4c439b1de77f4ac92749f55a
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [7]:
# wget ("curl") all files
user = 'FedorTaggenbrock'
repo = 'data_intensive_systems'
src_dir = "src"
pyfiles = ["clustering.py"]

for pyfile in pyfiles:
    url = f"https://raw.githubusercontent.com/{user}/{repo}/main/{src_dir}/{pyfile}"
    !wget --no-cache --backups=1 {url}


--2023-05-24 10:29:03--  https://raw.githubusercontent.com/FedorTaggenbrock/data_intensive_systems/main/src/clustering.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6256 (6.1K) [text/plain]
Saving to: ‘clustering.py’

clustering.py       100%[===================>]   6.11K  --.-KB/s    in 0s      

2023-05-24 10:29:04 (79.6 MB/s) - ‘clustering.py’ saved [6256/6256]



In [8]:
import clustering

**Test if this works**


In [9]:
res = clustering.jaccard_distance([1,1,0,1,0], [1,1,1,1,0])
print(res)

0.25


**Create a spark instance**

In [ ]:
spark_settings = {}
spark_instance = make_spark()

**Load and parse data**

In [ ]:
# Load and parse data
data_for_spark = parse_data()

In [ ]:
# Run clustering and parameter tuning
clustering_settings = {}
clustering_centroid_outcomes = run_clustering(spark_instance, data_for_spark, clustering_settings)

In [ ]:
# Evaluate clustering results
clustering_evaluation = evaluate_clustering(clustering_centroid_outcomes)

In [ ]:
# Display results (maybe only best result)
clustering_result = display_results(clustering_evaluation